# Import

In [ ]:
import pandas
import sklearn
import evaluation
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
GPU_fix = "MemoryFix" # Choose "Disable", "MemoryFix" or "None"

if GPU_fix == "Disable":
    try:
        # Disable all GPUS
        tf.config.set_visible_devices([], 'GPU')
        visible_devices = tf.config.get_visible_devices()
        for device in visible_devices:
            assert device.device_type != 'GPU'
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass
elif GPU_fix == "MemoryFix":
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

# Read training data

In [ ]:
folder = 'tau_data/'
train = pandas.read_csv(folder + 'training.csv', index_col='id')

In [4]:
train.head()

,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,...,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,production,signal,mass,min_ANNmuon
id,,,,,,,,,,,,,,,,,,,,,
18453471,0.001578,0.999999,14.033335,0.681401,0.016039,0.451886,1.900433,1482.037476,0.066667,0.060602,...,12290.760742,39264.398438,3.076006,4.003800,4.031514,458,-99,0,1866.300049,0.277559
5364094,0.000988,0.999705,5.536157,0.302341,0.142163,9.564503,0.865666,3050.720703,0.024022,0.019245,...,16562.667969,7341.257812,3.228553,2.786543,2.975564,406,-99,0,1727.095947,0.225924
11130990,0.000877,0.999984,6.117302,0.276463,0.034746,1.970751,10.975849,3895.908691,0.055044,0.047947,...,22695.388672,10225.309570,3.536903,2.865686,3.052810,196,-99,0,1898.588013,0.368630
15173787,0.000854,0.999903,5.228067,0.220739,0.076389,4.271331,3.276358,4010.781738,0.053779,0.006417,...,16909.515625,9141.426758,3.087461,3.218034,2.375592,137,-99,0,1840.410034,0.246045
1102544,0.001129,0.999995,39.069534,1.898197,0.120936,4.984982,0.468348,4144.546875,0.004491,0.037326,...,97612.804688,47118.785156,4.632295,4.711155,4.296878,477,-99,0,1899.793945,0.222060


In [41]:
# https://www.kaggle.com/sionek/ugbc-gs
train = pandas.read_csv(folder + 'training.csv', index_col='id')
test  = pandas.read_csv(folder + 'test.csv', index_col='id')


#--------------- feature engineering -------------- #
def add_features(df):
    # features used by the others on Kaggle
    df['NEW_FD_SUMP']=df['FlightDistance']/(df['p0_p']+df['p1_p']+df['p2_p'])
    df['NEW5_lt']=df['LifeTime']*(df['p0_IP']+df['p1_IP']+df['p2_IP'])/3
    df['p_track_Chi2Dof_MAX'] = df.loc[:, ['p0_track_Chi2Dof', 'p1_track_Chi2Dof', 'p2_track_Chi2Dof']].max(axis=1)
    #df['flight_dist_sig'] = df['FlightDistance']/df['FlightDistanceError'] # modified to:
    df['flight_dist_sig2'] = (df['FlightDistance']/df['FlightDistanceError'])**2
    # features from phunter
    df['flight_dist_sig'] = df['FlightDistance']/df['FlightDistanceError']
    df['NEW_IP_dira'] = df['IP']*df['dira']
    df['p0p2_ip_ratio']=df['IP']/df['IP_p0p2']
    df['p1p2_ip_ratio']=df['IP']/df['IP_p1p2']
    df['DCA_MAX'] = df.loc[:, ['DOCAone', 'DOCAtwo', 'DOCAthree']].max(axis=1)
    df['iso_bdt_min'] = df.loc[:, ['p0_IsoBDT', 'p1_IsoBDT', 'p2_IsoBDT']].min(axis=1)
    df['iso_min'] = df.loc[:, ['isolationa', 'isolationb', 'isolationc','isolationd', 'isolatione', 'isolationf']].min(axis=1)
    # My:
    # new combined features just to minimize their number;
    # their physical sense doesn't matter
    df['NEW_iso_abc'] = df['isolationa']*df['isolationb']*df['isolationc']
    df['NEW_iso_def'] = df['isolationd']*df['isolatione']*df['isolationf']
    df['NEW_pN_IP'] = df['p0_IP']+df['p1_IP']+df['p2_IP']
    df['NEW_pN_p']  = df['p0_p']+df['p1_p']+df['p2_p']
    df['NEW_IP_pNpN'] = df['IP_p0p2']*df['IP_p1p2']
    df['NEW_pN_IPSig'] = df['p0_IPSig']+df['p1_IPSig']+df['p2_IPSig']
    #My:
    # "super" feature changing the result from 0.988641 to 0.991099
    df['NEW_FD_LT']=df['FlightDistance']/df['LifeTime']
    return df

print("Add features")
train = add_features(train)
test = add_features(test)


print("Eliminate features")
filter_out = ['id', 'min_ANNmuon', 'production', 'mass', 'signal',
              'SPDhits','CDF1', 'CDF2', 'CDF3',
              'isolationb', 'isolationc','p0_pt', 'p1_pt', 'p2_pt',
              'p0_p', 'p1_p', 'p2_p', 'p0_eta', 'p1_eta', 'p2_eta',
              'isolationa', 'isolationb', 'isolationc', 'isolationd', 'isolatione', 'isolationf',
              'p0_IsoBDT', 'p1_IsoBDT', 'p2_IsoBDT',
              'p0_IP', 'p1_IP', 'p2_IP',
              'IP_p0p2', 'IP_p1p2',
              'p0_track_Chi2Dof', 'p1_track_Chi2Dof', 'p2_track_Chi2Dof',
              'p0_IPSig', 'p1_IPSig', 'p2_IPSig',
              'DOCAone', 'DOCAtwo', 'DOCAthree']

Add features
Eliminate features


# Define training features
Here we use subset of the all features to pass the agreement checking

In [7]:
#default
variables = ['LifeTime',
             'dira',
             'FlightDistance',
             'FlightDistanceError',
             'IP',
             'IPSig',
             'VertexChi2',
             'pt',
             'DOCAone',
             'DOCAtwo',
             'DOCAthree',
             'IP_p0p2',
             'IP_p1p2',
             'isolationa',
             'isolationb',
             'isolationc',
             'isolationd',
             'isolatione',
             'isolationf',
             'iso',
             'CDF1',
             'CDF2',
             'CDF3',
             'ISO_SumBDT',
             'p0_IsoBDT',
             'p1_IsoBDT',
             'p2_IsoBDT',
             'p0_track_Chi2Dof',
             'p1_track_Chi2Dof',
             'p2_track_Chi2Dof',
             'p0_IP',
             'p1_IP',
             'p2_IP',
             'p0_IPSig',
             'p1_IPSig',
             'p2_IPSig',
             'p0_pt',
             'p1_pt',
             'p2_pt',
             'p0_p',
             'p1_p',
             'p2_p',
             'p0_eta',
             'p1_eta',
             'p2_eta',
             'SPDhits',
             ]
print(len(variables))

46


In [29]:
#modified
variables = list(f for f in train.columns if f not in filter_out)
print(len(variables))

28


In [ ]:
#Normalizing dataset
df = train[variables]
train_mean = df.mean()
train_std = df.std()
train_df = (df - train_mean) / train_std

In [30]:
#alternative(batch_norm in model)
train_df = train[features]

In [ ]:

plt.figure();
df['DOCAone'].diff().hist(color="k", alpha=0.5, bins=1000);

In [19]:
train_df['LifeTime'].describe()

count    67552.000000
mean         0.001255
std          0.000779
min          0.000144
25%          0.000725
50%          0.001061
75%          0.001559
max          0.022134
Name: LifeTime, dtype: float64

# Baseline training

In [31]:
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(len(variables)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.05)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.05)), 
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.05)), 
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(50, activation=tf.keras.layers.LeakyReLU(alpha=0.05)),  
  tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
df_std = train_df.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(df.keys(), rotation=90)

In [ ]:
a = train_df["DOCAone"].to_numpy()
print(a.max())

In [ ]:
train_df[train_df["DOCAone"] > 20]

In [32]:
#Create training arrays
x_train = train_df.to_numpy()
y_train = train['signal'].to_numpy()
y_train = np.expand_dims(y_train,1)

In [33]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

In [34]:
history = model.fit(x_train, y_train, epochs=15, batch_size = 64)

Epoch 1/15
1056/1056 [==============================] - 5s 4ms/step - loss: 0.4005 - accuracy: 0.8278
Epoch 2/15
1056/1056 [==============================] - 5s 4ms/step - loss: 0.3784 - accuracy: 0.8372
Epoch 3/15
1056/1056 [==============================] - 5s 4ms/step - loss: 0.3690 - accuracy: 0.8411
Epoch 4/15
1056/1056 [==============================] - 4s 4ms/step - loss: 0.3651 - accuracy: 0.8424
Epoch 5/15
1056/1056 [==============================] - 5s 4ms/step - loss: 0.3651 - accuracy: 0.8433
Epoch 6/15
1056/1056 [==============================] - 5s 4ms/step - loss: 0.3639 - accuracy: 0.8438
Epoch 7/15
1056/1056 [==============================] - 5s 4ms/step - loss: 0.3605 - accuracy: 0.8438
Epoch 8/15
1056/1056 [==============================] - 5s 4ms/step - loss: 0.3609 - accuracy: 0.8440
Epoch 9/15
1056/1056 [==============================] - 5s 5ms/step - loss: 0.3647 - accuracy: 0.8431
Epoch 10/15
1056/1056 [==============================] - 5s 5ms/step - loss: 0.362

# Check agreement test

In [36]:
check_agreement = pandas.read_csv(folder + 'check_agreement.csv', index_col='id')
check_agreement = add_features(check_agreement)
agreement_probs = model.predict(check_agreement[variables].to_numpy()).squeeze()
print(agreement_probs)
ks = evaluation.compute_ks(
    agreement_probs[check_agreement['signal'].values == 0],
    agreement_probs[check_agreement['signal'].values == 1],
    check_agreement[check_agreement['signal'] == 0]['weight'].values,
    check_agreement[check_agreement['signal'] == 1]['weight'].values)
print('KS metric', ks, ks < 0.09)

[0.27458876 0.05568794 0.3891485  ... 0.8917255  0.982622   0.9739213 ]
KS metric 0.07655233476343726 True


# Check correlation test

In [37]:
check_correlation = pandas.read_csv(folder + 'check_correlation.csv', index_col='id')
check_correlation = add_features(check_correlation)
correlation_probs = model.predict(check_correlation[variables].to_numpy()).squeeze()
#pd.DataFrame(correlation_probs,
  #                 columns=[variables])
print(correlation_probs)
cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
print('CvM metric', cvm, cvm < 0.002)

[0.01726463 0.19499862 0.48506302 ... 0.78839487 0.37928677 0.02886738]
CvM metric 0.0010262050750394684 True


# Compute weighted AUC on the training data with min_ANNmuon > 0.4

In [38]:
train_eval = train[train['min_ANNmuon'] > 0.4]
train_probs = model.predict(train_eval[variables].to_numpy()).squeeze()
AUC = sklearn.metrics.roc_auc_score(train_eval['signal'], train_probs)
print('AUC', AUC)

AUC 0.9228560388071492


# Predict test, create file for kaggle

In [42]:
result = pandas.DataFrame({'id': test.index})
result['prediction'] = model.predict(test[variables].to_numpy()).squeeze()

In [43]:
result.to_csv('submission.csv', index=False, sep=',')

In [ ]:
!ls -l submission.csv

In [ ]:
result.to_csv?